# Chapter 6 - Prompt Engineering

## Loading our model

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
messages = [
    {"role": "user",
     "content": "Create a funny joke about chickens."}
]
output = pipe(messages)
output[0]['generated_text']

You are not running the flash-attention implementation, expect numerical differences.


' Why did the chicken join the band? Because it had the drumsticks!'

In [4]:
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
prompt

'<|user|>\nCreate a funny joke about chickens.<|end|>\n<|endoftext|>'

In [ ]:
output = pipe(messages, do_sample=True, temperature=1) # using high temperature for more randomness
print(output[0]['generated_text'])

 Why did the chicken join the school band? Because it wanted to beat the stereotype that it couldn't conduct!


In [6]:
output = pipe(messages, do_sample=True, top_p=1) # using top_p sampling
print(output[0]['generated_text'])

 Why did the chicken join the band? Because it had a "cluck" and rocked!


## Intro to Prompt Engineering
### The basic ingredients of a prompt
## Advanced Prompt Engineering
### Complex prompt

In [7]:
text = """In the previous post, we looked at Attention - a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer - a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neural Machine Translation model in specific tasks. The biggest benefit, however, comes from how The Transformer lends itself to parallelization. It is in fact Google Cloud's recommendation to use The Transformer as a reference model to use their Cloud TPU offering. So let's try to break the model apart and look at how it functions.
The Transformer was proposed in the paper Attention is All You Need. A TensorFlow implementation of it is available as a part of the Tensor2Tensor package. Harvard's NLP group created a guide annotating the paper with PyTorch implementation. In this post, we will attempt to oversimplify things a bit and introduce the concepts one by one to hopefully make it easier to understand to people without in-depth knowledge of the subject matter.
Let's begin by looking at the model as a single black box. In a machine translation application, it would take a sentence in one language, and output its translation in another.
Popping open that Optimus Prime goodness, we see an encoding component, a decoding component, and connections between them.
The encoding component is a stack of encoders (the paper stacks six of them on top of each other - there's nothing magical about the number six, one can definitely experiment with other arrangements). The decoding component is a stack of decoders of the same number.
The encoders are all identical in structure (yet they do not share weights). Each one is broken down into two sub-layers:
The encoder's inputs first flow through a self-attention layer - a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. We'll look closer at self-attention later in the post.
The outputs of the self-attention layer are fed to a feed-forward neural network. The exact same feed-forward network is independently applied to each position.
The decoder has both those layers, but between them is an attention layer that helps the decoder focus on relevant parts of the input sentence (similar what attention does in seq2seq models).
Now that we've seen the major components of the model, let's start to look at the various vectors/tensors and how they flow between these components to turn the input of a trained model into an output.
As is the case in NLP applications in general, we begin by turning each input word into a vector using an embedding algorithm.
Each word is embedded into a vector of size 512. We'll represent those vectors with these simple boxes.
The embedding only happens in the bottom-most encoder. The abstraction that is common to all the encoders is that they receive a list of vectors each of the size 512 - In the bottom encoder that would be the word embeddings, but in other encoders, it would be the output of the encoder that's directly below. The size of this list is hyperparameter we can set - basically it would be the length of the longest sentence in our training dataset.
After embedding the words in our input sequence, each of them flows through each of the two layers of the encoder.
Here we begin to see one key property of the Transformer, which is that the word in each position flows through its own path in the encoder. There are dependencies between these paths in the self-attention layer. The feed-forward layer does not have those dependencies, however, and thus the various paths can be executed in parallel while flowing through the feed-forward layer.
Next, we'll switch up the example to a shorter sentence and we'll look at what happens in each sub-layer of the encoder.
Now We're Encoding!
As we've mentioned already, an encoder receives a list of vectors as input. It processes this list by passing these vectors into a 'self-attention' layer, then into a feed-forward neural network, then sends out the output upwards to the next encoder.
"""

persona = "You are an expert in Large Language models. You excel at breaking down complex papers into digestible summaries.\n"
instruction = "Summarize the key findings of the paper provided.\n"
context = "Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.\n"
data_format = "Create a bullet-point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results.\n"
audience = "The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language models.\n"
tone = "The tone should be professional and clear.\n"
text = "MY TEXT TO SUMMARIZE"
data = f"Text to summarize: {text}"

query = persona + instruction + context + data_format + audience + tone + data

In [9]:
messages = [
    {"role": "user",
     "content": query}
]
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|user|>
You are an expert in Large Language models. You excel at breaking down complex papers into digestible summaries.
Summarize the key findings of the paper provided.
Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.
Create a bullet-point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results.
The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language models.
The tone should be professional and clear.
Text to summarize: MY TEXT TO SUMMARIZE<|end|>
<|endoftext|>


In [10]:
outputs = pipe(messages)
print(outputs[0]['generated_text'])

 - The paper investigates the impact of pre-training data size on the performance of Large Language Models (LLMs).

- It compares models trained on different volumes of data, ranging from a few billion to several tr


### In-Context Learning: Providing Examples

In [11]:
one_shot_prompt = [
    {
        "role": "user",
        "content": "A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word 'Gigamuru' is:"
    },
    {
        "role": "assistant",
        "content": "I have a Gigamuru that my uncle gave me as a gift. I love to play it at home."
    },
    {
        "role": "user",
        "content": "To 'screeg' something is to swing a sword at it. An example of a sentence that uses the word 'screeg' is:"
    }

]
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

<|user|>
A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word 'Gigamuru' is:<|end|>
<|assistant|>
I have a Gigamuru that my uncle gave me as a gift. I love to play it at home.<|end|>
<|user|>
To 'screeg' something is to swing a sword at it. An example of a sentence that uses the word 'screeg' is:<|end|>
<|endoftext|>


In [12]:
output = pipe(one_shot_prompt)
print(output[0]['generated_text'])

 In the ancient battle, the knight bravely screeg at the approaching enemy, hoping to protect his fellow warriors.


### Chain prompting: Breaking up the problem

In [13]:
Product_prompt = [
    {"role": "user", "content": "Create a name and slogan for a chatbot the leverages LLMs."}
]
outputs = pipe(Product_prompt)
print(outputs[0]['generated_text'])

 Name: ChatSage
Slogan: "Unleashing the power of AI to enhance your conversations."


## Reasoning with Generative Models

### Chain of Thought: Think before answering

In [16]:
standard_prompt = [
    {
        "role": "user",
        "content": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"
    },
    {
        "role": "assistant",
        "content": "11"
    },
    {
        "role": "user",
        "content": "The cafeteria had 25 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have now?"
    }
]
outputs = pipe(standard_prompt)
print(outputs[0]['generated_text'])

 The cafeteria started with 25 apples. They used 20 apples to make lunch, so they had 25 - 20 = 5 apples left. After buying 6 more apples, they now have 5 + 6 = 11 apples.


In [17]:
cot_prompt = [
    {
        "role": "user",
        "content": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"
    },
    {
        "role": "assistant",
        "content": "Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."
    },
    {
        "role": "user",
        "content": "The cafeteria had 25 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have now?"
    }
]
outputs = pipe(cot_prompt)
print(outputs[0]['generated_text'])

 The cafeteria started with 25 apples. They used 20 apples to make lunch, so they had 25 - 20 = 5 apples left. After buying 6 more apples, they now have 5 + 6 = 11 apples. The answer is 11.


### Zero-Shot Chain of Thought

In [18]:
zeroshot_cot_prompt = [
    {"role": "user", "content": "Imagine three different experts are answering this question. All experts will write down 1 step of their thinking, then share it with the group. Then all experts will go on to the next step, etc. If any expert realises they're wrong at any point then they leave. The question is 'The cafeteria had 25 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have now?' Make sure to discuss the results."}
]

In [21]:
outputs = pipe(zeroshot_cot_prompt)
print(outputs[0]['generated_text'])

 Expert 1:
Step 1: Start with the initial number of apples, which is 25.

Expert 2:
Step 1: Subtract the number of apples used for lunch, which is 20.
Step 2: Add the number of apples bought, which is 6.

Expert 3:
Step 1: Start with the initial number of apples, which is 25.
Step 2: Subtract the number of apples used for lunch, which is 20.
Step 3: Add the number of apples bought, which is 6.

Results:
All three experts arrived at the same answer: 11 apples. The cafeteria had 25 apples initially, used 20 for lunch, and then bought 6 more, leaving them with 11 apples.


## Output Verification

### Providing examples

In [22]:
zeroshot_prompt = [
    {"role": "user", "content": "Create a character profile for an RPG game in JSON format."}
]
outputs = pipe(zeroshot_prompt)
print(outputs[0]['generated_text'])

 ```json
{
  "name": "Eldrin the Wise",
  "race": "Elf",
  "class": "Wizard",
  "level": 10,
  "alignment": "Lawful Good",
  "strength": 8,
  "dexterity": 14,
  "constitution": 12,
  "intelligence": 18,
  "wisdom": 16,
  "charisma": 10,
  "weapon_skill": "simple",
  "armor_skill": "light",
  "spell_slots": {
    "cantrips": ["mage hand", "prestidigitation", "light", "comprehend languages", "detect magic"],
    "1st level": ["magic missile", "shield", "fire bolt", "lightning bolt", "eldritch blast"],
    "2nd level": ["invisibility", "polymorph"],
    "3rd level": ["fireball", "cure wounds", "shield", "detect thoughts"],
    "4th level": ["conjure elemental", "wish"],
    "5th level": ["true polymorph", "wish"]
  },
  "spells_per_day": 3,
  "equipment": {
    "weapon": "Staff of the Ancients",
    "armor": "Elven Robes",
    "accessories": ["Staff of Power", "Ring of Protection"],
    "spellbook": "The Arcane Compendium"
  },
  "background": "Adept",
  "personality": {
    "traits": ["C

In [23]:
one_shot_template = """Create a short character profile for an RPG game. Make sure to only use this format:

{
    "description": "A SHORT DESCRIPTION",
    "name": "THE CHARACTER NAME",
    "armor": "ONE PIECE OF ARMOR",
    "weapon": "ONE OR MORE WEAPONS"
}"""

one_shot_prompt = [
    {"role": "user", "content": one_shot_template}
]
outputs = pipe(one_shot_prompt)
print(outputs[0]['generated_text'])

 {
    "description": "A cunning rogue with a mysterious past, skilled in stealth and deception.",
    "name": "Shadowblade",
    "armor": "Leather Vest",
    "weapon": "Dagger, Crossbow"
}


### Grammar: Constrained Sampling

In [24]:
import gc
import torch
del model, tokenizer, pipe
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from llmma_